# Project: Investigating  FBI's gun dataset .

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#quest">Questions</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction ^_^

> In This Project We Will Be Analyzing Data Collected from the FBI's National Instant Criminal Background Check System.
<br>
<br>The data comes from the FBI's National Instant Criminal Background Check System. The NICS is used by to determine whether aprospective buyer is eligible to buy firearms or explosives. Gun shops call into this system to ensure that each customer does not have a criminal record or isn’t otherwise ineligible to make a purchase. The data has been supplemented with state level data from census.gov.



<a id='quest'></a>
## Questions !
><ol>
><li> Which states have had the highest and lowest gun per captia ?</li>
><li> Which states have had the highest growth in gun registrations ?</li>
><li> Which states have had the lowest growth in gun registrations ?</li>   
><li> What is the overall trend of gun purchases ?</li>
><li> What  is the relation between census data and high gun per capita?</li>
><ol>

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline 

<a id='wrangling'></a>
## Data Wrangling

### General Properties

__1. Loading our data and Performing operations to inspect data__

In [2]:
#loading census data and fbi gun data

In [ ]:
df_fbi = pd.read_excel("gun_data.xlsx")
df_census = pd.read_csv("U.S. Census Data.csv")

In [ ]:
#exploring data in both files

In [ ]:
df_fbi.head(3)

In [ ]:
df_census.head(3)

In [ ]:
df_fbi.shape

In [ ]:
df_census.shape

__2.types and look for instances of missing or possibly errant data.__

In [ ]:
#searching for problems in the data (Nan values ,dduplicated , wrong format , etc...)

In [ ]:
df_fbi.info()

In [ ]:
# to know type of data in month column

In [ ]:
type(df_fbi['month'][0])

In [ ]:
df_census.info()

In [ ]:
df_fbi.describe()

In [ ]:
df_census.describe()

In [ ]:
# to find number of states in fbi data

In [ ]:
df_fbi['state'].nunique()

In [ ]:
#to find number of states in census data 

In [ ]:
len(df_census.columns[2:])

In [ ]:
df_fbi.duplicated().sum()

In [ ]:
#number of duplicated rows in census data

In [ ]:
df_census.duplicated().sum()

__note>> there are 55 state in fbi_gun data but only 50 are in census data__

### Data Cleaning "

>__1. changing month column in fbi_gun data from string to date time.__

In [ ]:
# to change format of values in month column to (date time)

In [ ]:
df_fbi['month'] = pd.to_datetime(df_fbi['month']);

In [ ]:
#to make list of unique state's names

In [ ]:
uni = list(set(df_fbi['state']))

> __2.removing states which is not presented in the census data.__

In [ ]:
# to create list with states which is not pressented in census data but represented in fbi data

In [ ]:
nst = []
for state in uni:
    if state not in df_census.columns[2:]:       
        nst.append(state)
nst

In [ ]:
# to remove the states which is not represented in census data

In [ ]:
for i in nst:
    df_fbi = df_fbi[df_fbi.state != i]
df_fbi.state.nunique()

> __3.remving duplicated rows.__

In [ ]:
#to remove duplicated rows and makeing sure they are deleted

In [ ]:
df_census.drop_duplicates(inplace = True)
df_census.duplicated().sum()

> __4.removing seprator " , " from numbers__

In [ ]:
# converting numbers from 1000,000, format to 1000000 toavoid problems

In [ ]:
for i in df_census.iloc[1:,2:].columns:
    df_census[i] = df_census[i].str.replace(',', '')


> __5.extracting/converting numbers type from strings to floats.__

In [ ]:
# to convert string data in the data frame to float type

In [ ]:
for i in df_census.iloc[1:,2:].columns:
    df_census[i] = df_census[i].str.extract('(\d+)').astype(float)

In [ ]:
df_census.head()

In [ ]:
df_fbi['multiple'] = df_fbi['multiple'].astype(float)
df_fbi['totals'] = df_fbi['totals'].astype(float)

> __6.making sure if NAN values are missing data or just 0.0__

In [ ]:
#subtracting the sum of all guns from total column 
#we multiplied totals by 2 cause we put it to the summition first

In [ ]:
df_fbi[:].sum(axis=1)-2*df_fbi['totals']

### note>> the totals = sum of all columns so missing data are 0.0

> __7.convrting Nan values to 0.0__

In [ ]:
#to convert nan values to zeroes

In [ ]:
df_fbi.fillna(0.0, inplace = True)

In [ ]:
df_fbi.info()

In [ ]:
df_census.fillna(0, inplace = True)

In [ ]:
df_census.info()

In [ ]:
#converting columns and indexes to lower case to beasy to write and remember

In [ ]:
df_fbi['state'] = df_fbi['state'].str.lower()

In [ ]:
df_census.columns = df_census.columns.str.lower()

In [ ]:
df_census['fact'] = df_census['fact'].str.lower()

<a id='eda'></a>
## Exploratory Data Analysis

### Research Question 1 (Which states have had the highest and lowest gun per captia ?)

> __calculating highst and lowest and highst gun per captia in 2010__

In [ ]:
#extracting month and year from month column

In [ ]:
df_fbi['year'] = pd.DatetimeIndex(df_fbi['month']).year
df_fbi['month num'] = pd.DatetimeIndex(df_fbi['month']).month

In [ ]:
#making new data frame for 2010 data only

In [ ]:
df_2010 = df_fbi.query('year == 2010')

In [ ]:
# because data recorded in  census data in july only so we will combine guns data in 7 monthos only 

In [ ]:
df_2010 = df_2010[df_2010['month num'] <= 7]

In [ ]:
# index will be state to group our data

In [ ]:
df_2010 = df_2010.groupby('state')

In [ ]:
df_2010 = df_2010.sum()

In [ ]:
#making an array for gun per capita in 2010 and merging it with the data

In [ ]:
l = [] 
for i in df_2010.index:
    l.append(df_2010['totals'][i] / df_census[i][0])
df_2010['gun_per_capita'] = l

In [ ]:
# idxmax to find index with maximum vallue

In [ ]:
df_2010['gun_per_capita'].idxmax()

In [ ]:
df_2010['gun_per_capita'].loc[df_2010['gun_per_capita'].idxmax()]

In [ ]:
# idxmin to find index with minimum vallue

In [ ]:
df_2010['gun_per_capita'].idxmin()

In [ ]:
df_2010['gun_per_capita'].loc[df_2010['gun_per_capita'].idxmin()]

In [ ]:
#visualising our data

In [ ]:
a = df_2010['gun_per_capita'].plot(kind = 'bar',figsize=(18,10), title='gun per capita 2010');
a.set_xlabel("state", fontsize = 18);
a.set_ylabel("gun per capita", fontsize = 18);

## in 2010
### the highst gun per cpita in (kentucky) which equals 0.33466547245938333 
### the lowest gun per cpita in (new jersey) which equals 0.0031622894550811234

> __calculating highst and lowest and highst gun per captia in 2010__

In [ ]:
#doing the same with 2016 data 
##extracting month and year from month column

In [ ]:
df_fbi['year'] = pd.DatetimeIndex(df_fbi['month']).year
df_fbi['month num'] = pd.DatetimeIndex(df_fbi['month']).month

In [ ]:
df_2016 = df_fbi.query('year == 2016')

In [ ]:
# because data recorded in census data only so we will combine guns data in 7 monthos only 

In [ ]:
df_2016 = df_2016[df_2016['month num'] <= 7]

In [ ]:
df_2016 = df_2016.groupby('state')

In [ ]:
df_2016 = df_2016.sum()

In [ ]:
#making an array for gun per capita in 2010 and merging it with the data

In [ ]:
p = [] 
for i in df_2016.index:
    p.append(df_2016['totals'][i] / df_census[i][1])
df_2016['gun_per_capita'] = p

In [ ]:
df_2016['gun_per_capita'].idxmax()

In [ ]:
df_2016['gun_per_capita'].loc[df_2010['gun_per_capita'].idxmax()]

In [ ]:
df_2016['gun_per_capita'].idxmin()

In [ ]:
df_2016['gun_per_capita'].loc[df_2010['gun_per_capita'].idxmin()]

In [ ]:
#visualising our findings

In [ ]:
a = df_2016['gun_per_capita'].plot(kind = 'bar',figsize=(18,10), title='gun per capita 2016');
a.set_xlabel("state", fontsize = 18);
a.set_ylabel("gun per capita", fontsize = 18);

## in 2016
### the highst gun per cpita in (kentucky) which equals 0.51663845963813881
### the lowest gun per cpita in (hawaii) which equals 0.0080467900590460387

### Research Question 2  (Which states have had the highest growth in gun registrations ?)

In [ ]:
#to calculate in crease in gun per capita from 2010 to 2016

In [ ]:
df_2016['gun increase'] = df_2016['gun_per_capita']-df_2010['gun_per_capita']

In [ ]:
#visualising our data using bar chart

In [ ]:
df_2016['gun increase'].plot(kind = 'bar', figsize=(18,10), title='gun per capita growth from 2010 to 2016');
plt.ylabel("gun per capita increase", fontsize = 18);
plt.xlabel("state", fontsize = 18);

In [ ]:
df_2016['gun increase'].idxmax()

In [ ]:
df_2016['gun increase'].loc[df_2016['gun increase'].idxmax()]

####  NOTE >>kentucky  have the highest growth in gun registrations with an increase of 0.18197298717875549 in gun per capita from 2010 to 2016

### Research Question 3  (Which states have had the lowest growth in gun registrations ?)

In [ ]:
df_2016['gun increase'].idxmin()

In [ ]:
df_2016['gun increase'].loc[df_2016['gun increase'].idxmin()]

#### NOTE >> utah  have the lowest growth in gun registrations with an decrease of -0.033022139228623112 in gun per capita from 2010 to 2016

### Research Question 4  (What is the overall trend of gun purchases ?)

In [ ]:
#grouping data by month and year to observe the change in gun purchase

In [ ]:
trend = df_fbi.groupby(['month'])['totals'].sum()

In [ ]:
#Create line chart 

In [ ]:
trend.plot(kind='line', figsize=(18,12), label = 'Permit count');
plt.xlabel('Time',fontsize = 18);
plt.ylabel('gun purchase',fontsize = 18);
plt.title('gun purchase trending over time',fontsize = 18);

### Research Question 5 (What  is the relation between census data and high gun per capita?)

In [ ]:
z = [] 
for i in df_fbi.state.unique():
    z.append(df_census.iloc[0][i])
plt.figure(figsize=(10,10))
plt.scatter(z,df_2016['gun_per_capita']);
plt.ylabel("Gun_Per_Capital_2016");
plt.xlabel("population estimated in 2016");
plt.title("relation between gun per capita and population in 2016");



In [ ]:
z = [] 
for i in df_fbi.state.unique():
    z.append(df_census.iloc[1][i])
plt.figure(figsize=(10,10))
plt.scatter(z,df_2010['gun_per_capita']);
plt.ylabel("Gun_Per_Capital_2010");
plt.xlabel("population estimated in 2010");
plt.title("relation between gun per capita and population in 2010");


In [ ]:
z = [] 
for i in df_fbi.state.unique():
    z.append(df_census.iloc[22][i])
plt.figure(figsize=(10,10))
plt.scatter(z,df_2016['gun_per_capita']);
plt.ylabel("Gun_Per_Capital");
plt.xlabel("");
plt.title("relation between gun per capita and housing units");

In [ ]:
z = [] 
for i in df_fbi.state.unique():
    z.append(df_census.iloc[41][i])
plt.figure(figsize=(10,10))
plt.scatter(z,df_2016['gun_per_capita']);
plt.ylabel("Gun_Per_Capital_2016");
plt.xlabel("population estimated in 2016");
plt.title("relation between gun per capita and health care");

<a id='conclusions'></a>
## Conclusions
__1.Which states have had the highest and lowest gun per captia__
> in 2010
<br>the highst gun per cpita in (kentucky) which equals 0.33466547245938333 
<br>the lowest gun per cpita in (new jersey) which equals 0.0031622894550811234
><br> in 2016
<br>the highst gun per cpita in (kentucky) which equals 0.51663845963813881
<br>the lowest gun per cpita in (hawaii) which equals 0.0080467900590460387

__2.Which states have had the highest growth in gun registrations ?__
>kentucky  have the highest growth in gun registrations with an increase of 0.18197298717875549 in gun per capita from 2010 to 2016

__3.Which states have had the lowest growth in gun registrations ?__
>utah  have the lowest growth in gun registrations with an decrease of -0.033022139228623112 in gun per capita from 2010 to 2016

__4.the over all trend of gun purchase seems to be increasing over time.__

__5.there is just weak relation between census data and gun per capita.__


## Limitations
> __1.our data with separated in two files.
<br>2.there was some NAN values and some duplicated rows.
<br>3.there was some missing states data in census data.
<br>4.the data in census us was not sufficient as data was collected only in 2010 and 2016.
<br>5.census data wasn't orgganized at all

In [ ]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Investigate_a_Dataset.ipynb'])